In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import AlgoLoop
import StatisticFunc

plt.style.use('Solarize_Light2')                     # 绘图风格
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 字体雅黑
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False    # 处理负号

# 预处理

## 提取并处理数据

### 权益类资产换手率

In [ ]:
Turnovers = pd.read_excel(io="Raw/国内资产换手率.xlsx")                      # 读取数据
Turnovers = Turnovers.drop([0], axis=0)                                   # 删除多余行
Turnovers["日期"] = pd.DatetimeIndex(Turnovers["日期"])                    # 类型转换
Turnovers = Turnovers.set_index("日期")                                   # 设置索引

In [ ]:
Turnovers = Turnovers.drop(Turnovers.index[0], axis=0)                   # 日期对齐

### GC007利率

In [ ]:
Rates = pd.read_excel(io="Raw/GC007利率.xlsx")           #  读取数据
Rates = Rates.drop([0, 1, 2], axis=0)                   # 删除多余行
Rates.columns = ['日期', 'GC007']                        # 修改列名
Rates["日期"] = pd.DatetimeIndex(Rates["日期"])           # 类型转换
Rates = Rates.set_index("日期")                          # 设置索引

In [ ]:
Rates = Rates.drop(Rates.index[0], axis=0)                     # 日期对齐

### 十年国债收益率

In [ ]:
BondRates = pd.read_excel(io="Raw/中债国债到期收益率(中债)(日).xls")        # 读取数据
BondRates.columns = ['日期', '十年国债收益率']                             # 修改列名
BondRates = BondRates.drop([0, 3870, 3871], axis=0)                     # 删除多余行
BondRates["日期"] = pd.DatetimeIndex(BondRates["日期"])                   # 类型转换
BondRates = BondRates.set_index("日期")                                  # 设置索引

### 各资产收盘价

In [ ]:
Assets = pd.read_excel(io="Raw/国内资产收盘价.xlsx")
Assets = Assets.drop([0], axis=0)                                   # 删除多余行
Assets["日期"] = pd.DatetimeIndex(Assets["日期"])                    # 类型转换
Assets = Assets.set_index("日期")                                   # 设置索引

## 衍生数据生成

### 铜金价格比 & 铜油价格比

In [ ]:
Data = Assets.merge(BondRates, how='left', left_index=True, right_index=True)

In [ ]:
Data = Data.merge(Rates, how='left', left_index=True, right_index=True)

In [ ]:
Data['铜金价格比'] = Data['中信证券COMEX铜期货'] / Data['中信证券COMEX黄金期货']
Data['铜油价格比'] = Data['中信证券COMEX铜期货'] / Data['中信证券WTI原油期货']

### 十年国债价格指数

In [ ]:
Data['持有时长'] = (Data.index - Data.index[0]).days
Data['10年国债价格指数'] = 100 - (Data['十年国债收益率'] - 3.0) * 8.2 + 3.0 * Data['持有时长'] / 365.0
Data['10年国债价格指数'] = Data['10年国债价格指数'].fillna(method='ffill')

## 计算各资产收益率

### 日内损益

In [ ]:
# 利用自制10年国债价格指数，替换上证10年国债
Assets['上证10年国债'] = Data['10年国债价格指数'].values

In [ ]:
Returns = Assets.pct_change(axis=0)
Returns = Returns.dropna(axis=0, how='all')           # 删除无数据日

### 累计损益

In [ ]:
cumReturns = (1.0 + Returns)
cumReturns = cumReturns.fillna(1.0)                    # 填充空值
cumReturns = cumReturns.cumprod()                      # 计算各资产累计收益率

In [ ]:
cumReturns.plot(figsize=(16, 8))

# 风险平价 （指数权重EMA）

**资产明细**
- 沪深300指数
- 中证500指数
- 三年期3A信用债全价指数
- 十年期国债全价指数(自制)
- COMEX铜
- COMEX黄金
- NYMEX原油
- 铁矿石期货

回溯时间：2008年初-至今

买卖单笔手续费：千1

## 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]           # 调仓周期

NetValueDF_ema = pd.DataFrame(index=Returns.index)
DailyPnlDF_ema = pd.DataFrame(index=Returns.index) 
DrawDownDF_ema = pd.DataFrame(index=Returns.index)

for dtime in timeWindow:
    tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, Returns, cumReturns, Turnovers, mode='ema', dt=dtime)
    StatisticFunc.WeightPlot(tradeDF, weightDF, 'EMA'+str(dtime))
    
    NetValueDF_ema[str(dtime)+'组合净值'] = tradeDF['投资组合净值'] / 10000  
    DailyPnlDF_ema[str(dtime)+'组合日收益率'] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
    DrawDownDF_ema[str(dtime)+'最大回撤'] = tradeDF['最大回撤']                      

## 回测表现汇总

In [ ]:
# 汇总
summaryDF_ema = pd.DataFrame(index=['每月调仓', '每季度调仓', '每半年调仓', 
                                    '每三季度调仓', '每年调仓'])

summaryDF_ema["年平均收益率"] = DailyPnlDF_ema.mean(axis=0).values * 250
summaryDF_ema["年平均标准差"] = DailyPnlDF_ema.std(axis=0).values * np.sqrt(250)
summaryDF_ema["无基准夏普比率"] = summaryDF_ema["年平均收益率"] / summaryDF_ema["年平均标准差"]
summaryDF_ema["最大回撤"] = DrawDownDF_ema.min(axis=0).values
summaryDF_ema

## 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

for col in NetValueDF_ema.columns:
    plt.plot(NetValueDF_ema.index, NetValueDF_ema[col], lw=0.8, label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（指数平均）')
plt.show()

## 年间收益率分布

In [ ]:
yearReturns_ema = StatisticFunc.annualReturns(NetValueDF_ema)

In [ ]:
StatisticFunc.ReturnDist(yearReturns_ema, 2)

## 月间收益率分布

In [ ]:
monthlyReturns_ema = StatisticFunc.monthlyReturns(NetValueDF_ema)

In [ ]:
StatisticFunc.ReturnDist(monthlyReturns_ema, 2)

# 风险平价 + 杠杆策略（指数权重EMA）

## 杠杆调整后的债券净值

In [ ]:
lever = 2.0                                 # 杠杆
leverReturns = Returns.copy()               # 创建副本 
leverCumReturns = cumReturns.copy()   

# 更新杠杆调整后的债券净值
leverReturns.loc[:, ['上证10年国债', '信用债3-5AAA']] = Returns.loc[:, ['上证10年国债', '信用债3-5AAA']] * (1.0 + lever) - Rates.values * lever / (365.0 * 100)

# 更新累计净值
leverCumReturns = (1.0 + leverReturns)
leverCumReturns = leverCumReturns.fillna(1.0)                    # 填充空值
leverCumReturns = leverCumReturns.cumprod()                      # 计算各资产累计收益率

In [ ]:
leverReturns

## 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]           # 调仓周期

leverNetValueDF_ema = pd.DataFrame(index=Returns.index)
leverDailyPnlDF_ema = pd.DataFrame(index=Returns.index) 
leverDrawDownDF_ema = pd.DataFrame(index=Returns.index)

for dtime in timeWindow:
    tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', dt=dtime)
    StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆EMA'+str(dtime))
    
    leverNetValueDF_ema[str(dtime)+'组合净值'] = tradeDF['投资组合净值'] / 10000  
    leverDailyPnlDF_ema[str(dtime)+'组合日收益率'] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
    leverDrawDownDF_ema[str(dtime)+'最大回撤'] = tradeDF['最大回撤']                      

## 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_ema = pd.DataFrame(index=['每月调仓', '每季度调仓', '每半年调仓', 
                                    '每三季度调仓', '每年调仓'])

leverSummaryDF_ema["年平均收益率"] = leverDailyPnlDF_ema.mean(axis=0).values * 250
leverSummaryDF_ema["年平均标准差"] = leverDailyPnlDF_ema.std(axis=0).values * np.sqrt(250)
leverSummaryDF_ema["无基准夏普比率"] = leverSummaryDF_ema["年平均收益率"] / leverSummaryDF_ema["年平均标准差"]
leverSummaryDF_ema["最大回撤"] = leverDrawDownDF_ema.min(axis=0).values
leverSummaryDF_ema

## 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

for col in leverNetValueDF_ema.columns:
    plt.plot(leverNetValueDF_ema.index, leverNetValueDF_ema[col], lw=0.8, label=col)
    
# plt.plot(cumReturns.index, cumReturns['中证500'], lw=1.5, label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均）')
plt.show()

## 年间收益率分布

In [ ]:
yearReturns_ema = StatisticFunc.annualReturns(NetValueDF_ema)

In [ ]:
StatisticFunc.ReturnDist(yearReturns_ema, 2)

## 月间收益率分布

In [ ]:
monthlyReturns_ema = StatisticFunc.monthlyReturns(NetValueDF_ema)

In [ ]:
StatisticFunc.ReturnDist(monthlyReturns_ema, 2)

# 风险平价 + 杠杆 + 因子策略（指数权重EMA）

## 动量因子（横向比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]        # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':False, 
           'reverseX':False, 'reverseT':False, 
           'turnover':False,  
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds,
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆横截面动量（指数权重EMA）'+str(t+u))

        leverNetValueDF_momentumX.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000
        leverDailyPnlDF_momentumX.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_momentumX.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_momentumX = pd.DataFrame(index=multiIDX)

leverSummaryDF_momentumX["年平均收益率"] = leverDailyPnlDF_momentumX.mean(axis=0).values * 250
leverSummaryDF_momentumX["年平均标准差"] = leverDailyPnlDF_momentumX.std(axis=0).values * np.sqrt(250)
leverSummaryDF_momentumX["无基准夏普比率"] = leverSummaryDF_momentumX["年平均收益率"] / leverSummaryDF_momentumX["年平均标准差"]
leverSummaryDF_momentumX["最大回撤"] = leverDrawDownDF_momentumX.min(axis=0).values

leverSummaryDF_momentumX

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_momentumX.columns:
    plt.plot(leverNetValueDF_momentumX.index, leverNetValueDF_momentumX[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.5, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆横截面动量因子（指数权重EMA）')
plt.show()

## 动量因子（时序比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':True, 
           'reverseX':False, 'reverseT':False, 
           'turnover':False, 
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆时序动量（指数权重EMA）'+str(t+u))

        leverNetValueDF_momentumT.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_momentumT.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_momentumT.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_momentumT = pd.DataFrame(index=multiIDX)

leverSummaryDF_momentumT["年平均收益率"] = leverDailyPnlDF_momentumT.mean(axis=0).values * 250
leverSummaryDF_momentumT["年平均标准差"] = leverDailyPnlDF_momentumT.std(axis=0).values * np.sqrt(250)
leverSummaryDF_momentumT["无基准夏普比率"] = leverSummaryDF_momentumT["年平均收益率"] / leverSummaryDF_momentumT["年平均标准差"]
leverSummaryDF_momentumT["最大回撤"] = leverDrawDownDF_momentumT.min(axis=0).values

leverSummaryDF_momentumT

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_momentumT.columns:
    plt.plot(leverNetValueDF_momentumT.index, leverNetValueDF_momentumT[col], label=col)

plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')
    
plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆时间序列动量因子（指数权重EMA）')
plt.show()

## 反转因子（横向比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':True, 'reverseT':False, 
           'turnover':False,
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆横截面反转（指数权重EMA）'+str(t+u))

        leverNetValueDF_reverseX.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_reverseX.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_reverseX.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_reverseX = pd.DataFrame(index=multiIDX)

leverSummaryDF_reverseX["年平均收益率"] = leverDailyPnlDF_reverseX.mean(axis=0).values * 250
leverSummaryDF_reverseX["年平均标准差"] = leverDailyPnlDF_reverseX.std(axis=0).values * np.sqrt(250)
leverSummaryDF_reverseX["无基准夏普比率"] = leverSummaryDF_reverseX["年平均收益率"] / leverSummaryDF_reverseX["年平均标准差"]
leverSummaryDF_reverseX["最大回撤"] = leverDrawDownDF_reverseX.min(axis=0).values

leverSummaryDF_reverseX

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_reverseX.columns:
    plt.plot(leverNetValueDF_reverseX.index, leverNetValueDF_reverseX[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆横截面反转因子（指数权重EMA）')
plt.show()

## 反转因子 （时序比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':False, 'reverseT':True, 
           'turnover':False,
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆时序反转（指数权重EMA）'+str(t+u))

        leverNetValueDF_reverseT.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_reverseT.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_reverseT.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_reverseT = pd.DataFrame(index=multiIDX)

leverSummaryDF_reverseT["年平均收益率"] = leverDailyPnlDF_reverseT.mean(axis=0).values * 250
leverSummaryDF_reverseT["年平均标准差"] = leverDailyPnlDF_reverseT.std(axis=0).values * np.sqrt(250)
leverSummaryDF_reverseT["无基准夏普比率"] = leverSummaryDF_reverseT["年平均收益率"] / leverSummaryDF_reverseT["年平均标准差"]
leverSummaryDF_reverseT["最大回撤"] = leverDrawDownDF_reverseT.min(axis=0).values

leverSummaryDF_reverseT

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_reverseT.columns:
    plt.plot(leverNetValueDF_reverseT.index, leverNetValueDF_reverseT[col], label=col)

plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')
    
plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆时序反转因子（指数权重EMA）')
plt.show()

## 情绪因子（股指换手率）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':False, 'reverseT':False, 
           'turnover':True, 
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆换手率（指数权重EMA）'+str(t+u))

        leverNetValueDF_turnover.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_turnover.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_turnover.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_turnover = pd.DataFrame(index=multiIDX)

leverSummaryDF_turnover["年平均收益率"] = leverDailyPnlDF_turnover.mean(axis=0).values * 250
leverSummaryDF_turnover["年平均标准差"] = leverDailyPnlDF_turnover.std(axis=0).values * np.sqrt(250)
leverSummaryDF_turnover["无基准夏普比率"] = leverSummaryDF_turnover["年平均收益率"] / leverSummaryDF_turnover["年平均标准差"]
leverSummaryDF_turnover["最大回撤"] = leverDrawDownDF_turnover.min(axis=0).values

leverSummaryDF_turnover

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_turnover.columns:
    plt.plot(leverNetValueDF_turnover.index, leverNetValueDF_turnover[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆股指换手率因子（指数权重EMA）')
plt.show()

## 铜金价格比因子（十年国债反转）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 
            'copperGold':True, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆铜金价格比（指数权重EMA）'+str(t+u))

        leverNetValueDF_copperGold.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_copperGold.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_copperGold.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_copperGold = pd.DataFrame(index=multiIDX)

leverSummaryDF_copperGold["年平均收益率"] = leverDailyPnlDF_copperGold.mean(axis=0).values * 250
leverSummaryDF_copperGold["年平均标准差"] = leverDailyPnlDF_copperGold.std(axis=0).values * np.sqrt(250)
leverSummaryDF_copperGold["无基准夏普比率"] = leverSummaryDF_copperGold["年平均收益率"] / leverSummaryDF_copperGold["年平均标准差"]
leverSummaryDF_copperGold["最大回撤"] = leverDrawDownDF_copperGold.min(axis=0).values

leverSummaryDF_copperGold

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_copperGold.columns:
    plt.plot(leverNetValueDF_copperGold.index, leverNetValueDF_copperGold[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆铜金价格比因子（指数权重EMA）')
plt.show()

## 铜油价格比因子（沪深300动量）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 
            'copperGold':False, 'copperGas':True}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆铜油价格比（指数权重EMA）'+str(t+u))

        leverNetValueDF_copperGas.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_copperGas.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_copperGas.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_copperGas = pd.DataFrame(index=multiIDX)

leverSummaryDF_copperGas["年平均收益率"] = leverDailyPnlDF_copperGas.mean(axis=0).values * 250
leverSummaryDF_copperGas["年平均标准差"] = leverDailyPnlDF_copperGas.std(axis=0).values * np.sqrt(250)
leverSummaryDF_copperGas["无基准夏普比率"] = leverSummaryDF_copperGas["年平均收益率"] / leverSummaryDF_copperGas["年平均标准差"]
leverSummaryDF_copperGas["最大回撤"] = leverDrawDownDF_copperGas.min(axis=0).values

leverSummaryDF_copperGas

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_copperGas.columns:
    plt.plot(leverNetValueDF_copperGas.index, leverNetValueDF_copperGas[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆铜油价格比因子（指数权重EMA）')
plt.show()

# 风险平价+杠杆+多因子策略（指数平均EMA）

## 多因子（铜金 + 时序反转）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_multi = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_multi = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_multi = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':True,
            'turnover':False, 
            'copperGold':True, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆-铜金-时序反转（指数权重EMA）'+str(t+u))

        leverNetValueDF_multi.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_multi.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_multi.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_multi = pd.DataFrame(index=multiIDX)

leverSummaryDF_multi["年平均收益率"] = leverDailyPnlDF_multi.mean(axis=0).values * 250
leverSummaryDF_multi["年平均标准差"] = leverDailyPnlDF_multi.std(axis=0).values * np.sqrt(250)
leverSummaryDF_multi["无基准夏普比率"] = leverSummaryDF_multi["年平均收益率"] / leverSummaryDF_multi["年平均标准差"]
leverSummaryDF_multi["最大回撤"] = leverDrawDownDF_multi.min(axis=0).values

leverSummaryDF_multi

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_multi.columns:
    plt.plot(leverNetValueDF_multi.index, leverNetValueDF_multi[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆-铜金-时序反转（指数权重EMA）')
plt.show()

## 多因子（铜油 + 时序反转）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_multi2 = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_multi2 = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_multi2 = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':True,
            'turnover':False, 
            'copperGold':False, 'copperGas':True}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆-铜油-时序反转（指数权重EMA）'+str(t+u))

        leverNetValueDF_multi2.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_multi2.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_multi2.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_multi2 = pd.DataFrame(index=multiIDX)

leverSummaryDF_multi2["年平均收益率"] = leverDailyPnlDF_multi2.mean(axis=0).values * 250
leverSummaryDF_multi2["年平均标准差"] = leverDailyPnlDF_multi2.std(axis=0).values * np.sqrt(250)
leverSummaryDF_multi2["无基准夏普比率"] = leverSummaryDF_multi2["年平均收益率"] / leverSummaryDF_multi2["年平均标准差"]
leverSummaryDF_multi2["最大回撤"] = leverDrawDownDF_multi2.min(axis=0).values

leverSummaryDF_multi2

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_multi2.columns:
    plt.plot(leverNetValueDF_multi2.index, leverNetValueDF_multi2[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆-铜油-时序反转（指数权重EMA）')
plt.show()

# 最佳组合（杠杆+时序动量+铜金）

## 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_best = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_best = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_best = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':True, 'copperGas':True}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='ema', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆+横截面动量+时序动量+换手率+铜金+铜油（指数权重）'+str(t+u))

        leverNetValueDF_best.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_best.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_best.loc[:, (t, u)] = tradeDF['最大回撤']                      

## 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_best = pd.DataFrame(index=multiIDX)

leverSummaryDF_best["年平均收益率"] = leverDailyPnlDF_best.mean(axis=0).values * 250
leverSummaryDF_best["年平均标准差"] = leverDailyPnlDF_best.std(axis=0).values * np.sqrt(250)
leverSummaryDF_best["无基准夏普比率"] = leverSummaryDF_best["年平均收益率"] / leverSummaryDF_best["年平均标准差"]
leverSummaryDF_best["最大回撤"] = leverDrawDownDF_best.min(axis=0).values

leverSummaryDF_best

## 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_best.columns:
    plt.plot(leverNetValueDF_best.index, leverNetValueDF_best[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆+横截面动量+时序动量+换手率+铜金+铜油（指数权重）')
plt.show()